In [36]:
!pip install nrclex
!pip install nltk
!pip3 install text-preprocessing
!pip3 install pyspark
from nrclex import NRCLex
import pandas as pd
import nltk
import numpy as np
import re
import string
import math
nltk.download('punkt')
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
'Colab Notebooks'
'Information for personality profiling research.gform'
'Information for personality profiling research (Responses).gsheet'
'My Drive'
'Untitled folder'


In [37]:
data = pd.read_excel('/content/gdrive/MyDrive/Colab Notebooks/Emotion_Data.xlsx')

data.head()

,Age,Gender,Profession,Q1,Q2,Q3,Q4,Q5
0,23,Male,software quality assurance engineer,I'm pretty good at finding errors (in my every...,I love watching movies. Movies allow us to fee...,If i have the ability to change my personality...,i can keep my emotion by myself . and I am a v...,everyday life there are some stressful situati...
1,21,Female,Government job,Government job can ensure a better and securit...,I love reading books. Because it keeps my mind...,I am a very talkative person. I want ot minimi...,My patients is what makes me unique. I don't b...,"With my brain and prayers,and keeping patients..."
2,23,Male,Teaching,Teaching is a noble profession which comes wit...,"Yeah, outside work I do a lot of sports activi...",If I could change one thing about my personali...,My persevering make me unique.,Before my final defense I have faced heavy str...
3,28,Male,Teaching,I have to make lots of money by getting into t...,I like to conflict with others.,I am a very emotional person. I want to hold m...,My patients is what makes me unique. I don't b...,Once I faced my crush that was the most stress...
4,26,Male,Pharmacist,For good carrier. I like this job .,Cricket. Its helps me to get rid of stress.,I have to be more cruel,I have great Personality,"That was my first presentation in a job, it wa..."


In [38]:
data= data.drop(["Age","Profession","Gender"],axis=1)
data.head()

,Q1,Q2,Q3,Q4,Q5
0,I'm pretty good at finding errors (in my every...,I love watching movies. Movies allow us to fee...,If i have the ability to change my personality...,i can keep my emotion by myself . and I am a v...,everyday life there are some stressful situati...
1,Government job can ensure a better and securit...,I love reading books. Because it keeps my mind...,I am a very talkative person. I want ot minimi...,My patients is what makes me unique. I don't b...,"With my brain and prayers,and keeping patients..."
2,Teaching is a noble profession which comes wit...,"Yeah, outside work I do a lot of sports activi...",If I could change one thing about my personali...,My persevering make me unique.,Before my final defense I have faced heavy str...
3,I have to make lots of money by getting into t...,I like to conflict with others.,I am a very emotional person. I want to hold m...,My patients is what makes me unique. I don't b...,Once I faced my crush that was the most stress...
4,For good carrier. I like this job .,Cricket. Its helps me to get rid of stress.,I have to be more cruel,I have great Personality,"That was my first presentation in a job, it wa..."


In [39]:
data["Text"] = data["Q1"].astype(str)+' '+ data["Q2"].astype(str)+' '+data["Q3"].astype(str)+' '+data["Q4"].astype(str)+' '+ data["Q5"]
data= data.drop(["Q1","Q2","Q3","Q4","Q5"],axis=1)
data.head()

,Text
0,I'm pretty good at finding errors (in my every...
1,Government job can ensure a better and securit...
2,Teaching is a noble profession which comes wit...
3,I have to make lots of money by getting into t...
4,For good carrier. I like this job . Cricket. I...


In [40]:
data.drop_duplicates(inplace=True)

In [41]:
contraction_dict = contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

# Regular expression for finding contractions
contractions_re=re.compile('(%s)' % '|'.join(contraction_dict.keys()))
def expand_contractions(text,contractions_dict=contraction_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)
# Expanding Contractions in the reviews


In [42]:
data=data.astype(str)
data['Text'] = data['Text'].str.lower()

data['Text'] = data['Text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))
data

,Text
0,im pretty good at finding errors in my everyda...
1,government job can ensure a better and securit...
2,teaching is a noble profession which comes wit...
3,i have to make lots of money by getting into t...
4,for good carrier i like this job cricket its ...
...,...
96,i want to be famous i like to cook it gives me...
97,gaming is my passion singing various song make...
98,i want to serve my country i like to train dog...
99,i like to capture moment i like to collect coi...


In [43]:
#remove stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words.add('subject')
stop_words.add('http')
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])
data['Text'] = data['Text'].apply(lambda x: remove_stopwords(x))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [44]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
data['Text']= data['Text'].apply(lambda text: lemmatize_words(text))
data

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Text
0,im pretty good finding error everyday life ins...
1,government job ensure better security life maj...
2,teaching noble profession come much responsibi...
3,make lot money getting profession like conflic...
4,good carrier like job cricket help get rid str...
...,...
96,want famous like cook give joy want avoid lazi...
97,gaming passion singing various song make happy...
98,want serve country like train dogit make happy...
99,like capture moment like collect coin differen...


In [45]:
data['Text'] = data['Text'].apply(lambda text: re.sub(' +', ' ', text))

In [46]:
data["emotions"]=(data['Text']).apply(lambda x: NRCLex(x).affect_frequencies)
data

,Text,emotions
0,im pretty good finding error everyday life ins...,"{'fear': 0.08433734939759036, 'anger': 0.03614..."
1,government job ensure better security life maj...,"{'fear': 0.08196721311475409, 'anger': 0.03278..."
2,teaching noble profession come much responsibi...,"{'fear': 0.08571428571428572, 'anger': 0.05714..."
3,make lot money getting profession like conflic...,"{'fear': 0.05555555555555555, 'anger': 0.16666..."
4,good carrier like job cricket help get rid str...,"{'fear': 0.0625, 'anger': 0.0625, 'anticip': 0..."
...,...,...
96,want famous like cook give joy want avoid lazi...,"{'fear': 0.2222222222222222, 'anger': 0.111111..."
97,gaming passion singing various song make happy...,"{'fear': 0.03125, 'anger': 0.0, 'anticip': 0.0..."
98,want serve country like train dogit make happy...,"{'fear': 0.05555555555555555, 'anger': 0.0, 'a..."
99,like capture moment like collect coin differen...,"{'fear': 0.0, 'anger': 0.14285714285714285, 'a..."


In [47]:
data = pd.concat([data.drop(['emotions'], axis = 1), data['emotions'].apply(pd.Series)], axis = 1)
data.head()

,Text,fear,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy,anticipation
0,im pretty good finding error everyday life ins...,0.084337,0.036145,0.0,0.144578,0.072289,0.216867,0.108434,0.072289,0.012048,0.144578,0.108434
1,government job ensure better security life maj...,0.081967,0.032787,0.0,0.081967,0.049180,0.360656,0.098361,0.049180,0.016393,0.098361,0.131148
2,teaching noble profession come much responsibi...,0.085714,0.057143,0.0,0.200000,0.057143,0.400000,0.028571,0.000000,0.000000,0.085714,0.085714
3,make lot money getting profession like conflic...,0.055556,0.166667,0.0,0.055556,0.111111,0.222222,0.111111,0.055556,0.055556,0.055556,0.111111
4,good carrier like job cricket help get rid str...,0.062500,0.062500,0.0,0.062500,0.125000,0.250000,0.125000,0.062500,0.062500,0.062500,0.125000


In [48]:
data= data.drop(["Text"],axis=1)
data.head()

,fear,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy,anticipation
0,0.084337,0.036145,0.0,0.144578,0.072289,0.216867,0.108434,0.072289,0.012048,0.144578,0.108434
1,0.081967,0.032787,0.0,0.081967,0.049180,0.360656,0.098361,0.049180,0.016393,0.098361,0.131148
2,0.085714,0.057143,0.0,0.200000,0.057143,0.400000,0.028571,0.000000,0.000000,0.085714,0.085714
3,0.055556,0.166667,0.0,0.055556,0.111111,0.222222,0.111111,0.055556,0.055556,0.055556,0.111111
4,0.062500,0.062500,0.0,0.062500,0.125000,0.250000,0.125000,0.062500,0.062500,0.062500,0.125000


In [49]:
data = data.astype(float)
data = round(data,4)
data.head()

,fear,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy,anticipation
0,0.0843,0.0361,0.0,0.1446,0.0723,0.2169,0.1084,0.0723,0.0120,0.1446,0.1084
1,0.0820,0.0328,0.0,0.0820,0.0492,0.3607,0.0984,0.0492,0.0164,0.0984,0.1311
2,0.0857,0.0571,0.0,0.2000,0.0571,0.4000,0.0286,0.0000,0.0000,0.0857,0.0857
3,0.0556,0.1667,0.0,0.0556,0.1111,0.2222,0.1111,0.0556,0.0556,0.0556,0.1111
4,0.0625,0.0625,0.0,0.0625,0.1250,0.2500,0.1250,0.0625,0.0625,0.0625,0.1250


In [51]:
path= "/content/gdrive/MyDrive/Colab Notebooks/Emotion_Raw_Data.xlsx"
with pd.ExcelWriter(path) as writer:
     data.to_excel(writer) 

In [52]:
data

,fear,anger,anticip,trust,surprise,positive,negative,sadness,disgust,joy,anticipation
0,0.0843,0.0361,0.0,0.1446,0.0723,0.2169,0.1084,0.0723,0.0120,0.1446,0.1084
1,0.0820,0.0328,0.0,0.0820,0.0492,0.3607,0.0984,0.0492,0.0164,0.0984,0.1311
2,0.0857,0.0571,0.0,0.2000,0.0571,0.4000,0.0286,0.0000,0.0000,0.0857,0.0857
3,0.0556,0.1667,0.0,0.0556,0.1111,0.2222,0.1111,0.0556,0.0556,0.0556,0.1111
4,0.0625,0.0625,0.0,0.0625,0.1250,0.2500,0.1250,0.0625,0.0625,0.0625,0.1250
...,...,...,...,...,...,...,...,...,...,...,...
96,0.2222,0.1111,0.0,0.0000,0.0000,0.2222,0.2222,0.1111,0.0000,0.1111,NaN
97,0.0312,0.0000,0.0,0.2188,0.0938,0.2188,0.0312,0.0312,0.0000,0.1875,0.1875
98,0.0556,0.0000,0.0,0.2778,0.0556,0.3333,0.0556,0.0000,0.0000,0.0556,0.1667
99,0.0000,0.1429,0.0,0.1429,0.0000,0.1429,0.2857,0.0000,0.0000,0.1429,0.1429
